In [1]:
import os
import imp
import numpy as np
import pandas as pd
import functions as utils

from sklearn.model_selection import train_test_split
from functools import partial
from concurrent.futures import ProcessPoolExecutor, Executor, as_completed

In [2]:
def read_data(path, filename):
    print(f'reading file = {os.path.join(path, filename)}')
    data = pd.read_csv(os.path.join(path, filename))
    data = data.rename(columns={'Unnamed: 0':'Id'})
    print(f'data shape = {data.shape}')
    types_info = pd.DataFrame(data.dtypes.value_counts(), columns=['columns_count'])
    print('types info about df columns: ')
    print(types_info)
    return data

In [3]:
# ls -la -h Datasets/GiveMeSomeCredit/

In [3]:
train_data = read_data(path='Datasets/GiveMeSomeCredit/', filename='cs-training.csv')
# test_data = read_data(path='Datasets/GiveMeSomeCredit/', filename='cs-test.csv')
# descript = pd.read_excel("Datasets/GiveMeSomeCredit/Data Dictionary.xls")
# sample_data = pd.read_csv("Datasets/GiveMeSomeCredit/sampleEntry.csv")

reading file = Datasets/GiveMeSomeCredit/cs-training.csv
data shape = (150000, 12)
types info about df columns: 
         columns_count
int64                8
float64              4


In [4]:
train_data.fillna(0, inplace=True)
# test_data.fillna(0, inplace=True)

#ставим колонку Id как индекс клиента
train_data.set_index('Id', inplace=True)
# test_data.set_index('Id', inplace=True)

#сохраняем метку класса
train_label = train_data['SeriousDlqin2yrs'].copy()
train_data.drop('SeriousDlqin2yrs', axis=1, inplace=True)
#удаляем колонку класса из тестовых данных, так как она не несет никакой информации
# test_data.drop('SeriousDlqin2yrs', axis=1, inplace=True)

In [5]:
train_data['NumberOfDependents'] = train_data.NumberOfDependents.astype('int')
train_data['MonthlyIncome'] = train_data.MonthlyIncome.astype('int')

In [6]:
train_data.shape#, test_data.shape

(150000, 10)

In [7]:
float_cols = train_data.select_dtypes('float').columns
train_data.loc[:, float_cols] = train_data.loc[:, float_cols].round(2)
# train_data.head()

In [8]:
# float_cols = test_data.select_dtypes('float').columns
# test_data.loc[:, float_cols] = test_data.loc[:, float_cols].round(2)
# test_data.head()

In [9]:
transformed_train = utils.transform_to_description(train_data)
# transformed_test = transform_to_description(test_data)
# transformed_train.head(5)

In [10]:
transformed_train.shape#, transformed_test.shape

(150000, 10)

In [11]:
trainX, testX, trainY, testY = train_test_split(transformed_train, train_label, test_size=0.4)

In [12]:
valX, testX, valY, testY = train_test_split(testX, testY, test_size=0.5)

In [13]:
print(trainX.shape, valX.shape, testX.shape)
print(trainY.shape, valY.shape, testY.shape)

(90000, 10) (30000, 10) (30000, 10)
(90000,) (30000,) (30000,)


**Алгоритм из работы Алексея(QBCA)**

In [14]:
# alpha = 0.005
# sample_ratio = 0.01
# num_iters = 100
# N_neg = train_label.value_counts().reset_index().iloc[0, 1]
# N_pos = train_label.value_counts().reset_index().iloc[1, 1]
# N_neg, N_pos

**не обновляем индекс так как индекс - это id клиента, имеет значимую информацию**

In [19]:
# train_pos = transformed_train.loc[train_label[train_label == 1].index]
# train_neg = transformed_train.loc[train_label[train_label == 0].index]
train_pos = trainX.loc[trainY[trainY == 1].index]
train_neg = trainX.loc[trainY[trainY == 0].index]

train_neg.shape, train_pos.shape

((83940, 10), (6060, 10))

**Берем выборку заемов, для начального тестирования работоспособности**

In [20]:
train_pos = train_pos.sample(n=1000, replace=False)
train_neg = train_neg.sample(n=1000, replace=False)
train_pos.shape, train_neg.shape

((1000, 10), (1000, 10))

In [21]:
test_sample = valX.sample(1)
test_sample

,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
Id,,,,,,,,,,
113187,"(1.0, 1.0)","(44, 44)","(0, 0)","(0.13, 0.13)","(5700, 5700)","(3, 3)","(0, 0)","(1, 1)","(0, 0)","(2, 2)"


In [25]:
train_pos

,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
Id,,,,,,,,,,
47663,"(0.78, 0.78)","(36, 36)","(1, 1)","(2.98, 2.98)","(1365, 1365)","(7, 7)","(1, 1)","(0, 0)","(2, 2)","(1, 1)"
105701,"(0.06, 0.06)","(65, 65)","(1, 1)","(0.64, 0.64)","(12250, 12250)","(24, 24)","(0, 0)","(13, 13)","(0, 0)","(1, 1)"
87670,"(1.0, 1.0)","(27, 27)","(0, 0)","(0.0, 0.0)","(2000, 2000)","(0, 0)","(1, 1)","(0, 0)","(1, 1)","(0, 0)"
50310,"(0.88, 0.88)","(78, 78)","(3, 3)","(423.0, 423.0)","(0, 0)","(4, 4)","(0, 0)","(1, 1)","(0, 0)","(0, 0)"
50047,"(1.0, 1.0)","(51, 51)","(1, 1)","(0.62, 0.62)","(6058, 6058)","(8, 8)","(0, 0)","(2, 2)","(1, 1)","(0, 0)"
53256,"(1.02, 1.02)","(41, 41)","(1, 1)","(0.53, 0.53)","(4198, 4198)","(5, 5)","(1, 1)","(1, 1)","(1, 1)","(2, 2)"
123640,"(0.88, 0.88)","(62, 62)","(0, 0)","(0.52, 0.52)","(3000, 3000)","(10, 10)","(0, 0)","(2, 2)","(0, 0)","(0, 0)"
75586,"(1.0, 1.0)","(67, 67)","(2, 2)","(706.0, 706.0)","(0, 0)","(2, 2)","(3, 3)","(0, 0)","(2, 2)","(0, 0)"
28913,"(1.14, 1.14)","(25, 25)","(0, 0)","(102.0, 102.0)","(0, 0)","(1, 1)","(0, 0)","(0, 0)","(0, 0)","(0, 0)"


### Mining step


    для положительного класса нас интересуют объекты отрицательного класса, 
    а для отрицательного - положительные
    Уже после определения объектов из другого класса, попадающие в признаковое представление семпла данных, будет приниматься решение
     о включение этого признакого представление в список гипотез представления(областей или интервальных представлений)
    

Что нужно посмотреть:

    1. Сравнение количества генерируемых гипотез в зависимости от критерия и области генерации выборки.
    (Возможно изобразить всего 4 варианта: старый подход - локальная или случайная выборка, новый подход - 
    локальная или случайная выборка)
    
    2. Сравнение генерируемых гипотез для старого и нового критерия в зависимости от ширины локальной области генерации
    (Возможно есть какая то оптимальная ширина окна)
    

To-do-list:
1. **классификации при разном включении признаков из исходного множества**

2. **Подумать о том, каким образом генерить гипотезы. То есть выбирать не случайно выборку из множества объектов,
    а какую-то локальную область. (Делать будем через расширение области объекта; более того, будем искать 
    оптимальное значение расширения локальной области) done !!!**

In [24]:
utils = imp.reload(utils)

In [89]:
def generate_hypothesis(iteration: int, obj: pd.Series, object_area: pd.Series, train_data: pd.DataFrame, 
                        other_data: pd.DataFrame, sample_size: int, hypothesis_criterion: str,
                        sample_type:str, verbose: bool, alpha: float):
        
        print(f'iteration: {iteration}')
        
        if sample_type == 'local' and object_area is None:
            print(f'Cannot generate sample from local area. Got None as local object area param!')
            raise NotImplementedError('Wrong params for local sampling!')
    
        if sample_type == 'random' and object_area is not None:
            print(f'got misleading params values. Got sample_type = None and local object area is not None')
            return NotImplementedError('Wrong params for random sampling!')

        #generating random sample using sample strategy = {sample_type}
        sample = utils.generate_random_sample(train_data=train_data, sample_size=sample_size, d=object_area) 
        sample.append(obj)
        
        d = utils.get_similarity_sample_repr(sample)
        if verbose:
            print('got feature represantation for sample')
        
        d_other_objects = utils.is_included_in_repr(d, train_data=other_data)
#         if d_other_objects is not None:
#             print(f'got {d_other_objects.shape[0]} d_other_objects')
#             print(f'thresh for hypothesis = {int(other_data.shape[0] * alpha)}')
    
        if d_other_objects is None:
            return d    
        
        result_hypothesis = utils.check_criterion(d=d, train_data=train_data, 
                                                  hypothesis_criterion=hypothesis_criterion, 
                                                  d_other_objects=d_other_objects,
                                                  other_data=other_data, alpha=alpha
                                                 )

        return result_hypothesis
        

        

def mining_step(test_obj: pd.Series,train_pos: pd.DataFrame, train_neg: pd.DataFrame, num_iters: int, 
                sample_ratio: float, alpha: float, hypothesis_criterion: str, sample_type: str,
                mining_type: str = 'pos', verbose : bool = False, n_jobs : int = 4):
    """
    hypothesis_criterion: 'contr_class', если используем базовый критерий, 
                                когда смотрится пересечение с противоположным классом(старый критерий отбора гипотез)
                           'both_classes', когда интересует пересечение по обоим классам(новый критерий отбора гипотез)
                           
    sample_type: 'random', если берем произвольную выборку интервальных представлений
                 'local', если берем произвольную выборку из локальной области
    
    returns list of hypothesises
    """
    
    if sample_type == 'local':
        #sampling
        print(f'start searching optimal local area')
        object_area = utils.find_opt_local_area(obj=test_obj,
                                                train_data=train_data,
                                                frac=0.15,
                                                num_iters=30
                                               )
    else:
        print(f'using random sample from train data')
        object_area = None
    
    train_data = train_pos if mining_type == 'pos' else train_neg
    other_data = train_neg if mining_type == 'pos' else train_pos
    sample_size = int(train_data.shape[0] * sample_ratio)
    print('start generating hypothesises')
    
    mining = partial(generate_hypothesis, 
                     obj=test_obj, 
                     object_area=object_area,
                     train_data=train_data, 
                     other_data=other_data, 
                     sample_size=sample_size, 
                     hypothesis_criterion=hypothesis_criterion,
                     sample_type=sample_type,
                     verbose=verbose,
                     alpha=alpha
                    )
    
    with ProcessPoolExecutor(max_workers=n_jobs) as executor:
        hypothesises = executor.map(mining, range(num_iters))

    hypothesises = [res for res in hypothesises if res is not None]
    print(f'All hypothesises are generated!')
    return hypothesises

In [27]:
# for i, obj in test_sample.iterrows():
#     area = utils.find_opt_local_area(obj=obj, train_data=train_pos, num_iters=100)
#     if area is not None:
#         print('we found area')

In [50]:
# generate_local_sample(d=area, train_data=train_pos, sample_size=int(sample_ratio * train_pos.shape[0]))

In [51]:
# local_neg_objs = is_included_in_repr(d=area, train_data=train_neg)
# local_objs = is_included_in_repr(d=area, train_data=train_pos)
# len(local_objs), train_pos.shape, len(local_neg_objs)

In [179]:
# for i, obj in test_sample.iterrows():
#     sample = generate_local_sample(obj, train_data=train_pos, sample_size=3, num_iters=30)

In [183]:
# for i, obj in test_sample.iterrows():
#     print(type(obj))
# #     generate_local_sample(obj=obj, train_data=train_pos, sample_size=3)
#     generated = generate_local_area(obj)

In [55]:
# test_local_obj = transformed_train.sample(1)
# test_local_obj

# generated_obj = generate_local_area(test_local_obj)
# generated_obj

# generate_local_area(generated_obj)

In [56]:
train_pos.shape, train_neg.shape

((1000, 10), (1000, 10))

In [93]:
alpha = 0.001
sample_ratio = 0.002
num_iters = 1000

In [38]:
alpha_params = [0.000, 0.001, 0.002]#, 0.003, 0.004]
sample_ratio_params = [0.001, 0.002, 0.004]#, 0.006, 0.008]
alpha_params, sample_ratio_params

([0.0, 0.001, 0.002], [0.001, 0.002, 0.004])

In [ ]:
!!!Нужно построить график зависимости генерации гипотез от количества параметров
Нужно думаю что минимум 10 итераций

In [90]:
results_pos = {}
results_neg = {}
pos_hyps = []
neg_hyps = []

In [91]:
num_iters = 1000

In [295]:
%%time
for i, obj in test_sample.iterrows():
    for alpha in alpha_params:
        for sample_ratio in sample_ratio_params:
            print(f'using params: alpha = {alpha}, sample_ratio = {sample_ratio}, num_iters = {num_iters}')
            
            pos_hyps = mining_step(test_obj=obj, train_pos=train_pos, train_neg=train_neg, 
                           num_iters=num_iters,sample_ratio=sample_ratio, alpha = alpha,
                           hypothesis_criterion='contr_class',
                           sample_type='random',
                           mining_type='pos',
                           verbose=False, n_jobs=4
                          )
            pos_hyps_shape = pd.DataFrame(pos_hyps).shape[0] if len(pos_hyps) > 0 else 0
            results_pos[(alpha, sample_ratio, num_iters)] = pos_hyps_shape
#             print(f'start mining from neg objects')
            neg_hyps = mining_step(test_obj=obj, train_pos=train_pos, train_neg=train_neg,
                                   num_iters=num_iters, sample_ratio=sample_ratio, alpha = alpha, 
                                   hypothesis_criterion='contr_class',mining_type='neg', 
                                   sample_type='random',
                                   verbose=False, n_jobs=4
                                  )
            neg_hyps_shape = pd.DataFrame(neg_hyps).shape[0] if len(neg_hyps) > 0 else 0
            results_neg[(alpha, sample_ratio, num_iters)] = neg_hyps_shape

In [45]:
results_poss

{(0.0, 0.001, 1000): 1000,
 (0.0, 0.002, 1000): 608,
 (0.0, 0.004, 1000): 17,
 (0.001, 0.001, 1000): 1000,
 (0.001, 0.002, 1000): 740,
 (0.001, 0.004, 1000): 37,
 (0.002, 0.001, 1000): 1000,
 (0.002, 0.002, 1000): 790,
 (0.002, 0.004, 1000): 51}

In [43]:
results_neg

{(0.0, 0.001, 1000): 1000,
 (0.0, 0.002, 1000): 601,
 (0.0, 0.004, 1000): 11,
 (0.001, 0.001, 1000): 1000,
 (0.001, 0.002, 1000): 754,
 (0.001, 0.004, 1000): 46,
 (0.002, 0.001, 1000): 1000,
 (0.002, 0.002, 1000): 826,
 (0.002, 0.004, 1000): 57}

In [35]:
results_pos

{(0.0, 0.001, 100): 100,
 (0.0, 0.002, 100): 58,
 (0.001, 0.001, 100): 100,
 (0.001, 0.002, 100): 76}

In [36]:
results_neg

{(0.0, 0.001, 100): 100,
 (0.0, 0.002, 100): 66,
 (0.001, 0.001, 100): 100,
 (0.001, 0.002, 100): 75}

**Генерация гипотез для полож класса занимает 10 мин (с 4 процессами) c 3000 iterations**
**c 1000 iterations - 2 mins**

In [294]:
# %%time
# for i, obj in test_sample.iterrows():
#     print(type(obj))
#     print(f'start mining from pos objects')
#     pos_hyps = mining_step(test_obj=obj, train_pos=train_pos, train_neg=train_neg, 
#                            num_iters=num_iters,sample_ratio=sample_ratio, alpha = alpha,
#                            hypothesis_criterion='contr_class',
#                            sample_type='random',
#                            mining_type='pos',
#                            verbose=False, n_jobs=4
#                           )
    
#     print(f'start mining from neg objects')
#     neg_hyps = mining_step(test_obj=obj, train_pos=train_pos, train_neg=train_neg,
#                            num_iters=num_iters, sample_ratio=sample_ratio, alpha = alpha, 
#                            hypothesis_criterion='contr_class',mining_type='neg', 
#                            sample_type='random',
#                            verbose=False, n_jobs=4
#                           )

In [95]:
len(pos_hyps), len(neg_hyps)

(731, 780)

In [104]:
pos_hyps = pd.DataFrame(pos_hyps)

In [105]:
neg_hyps = pd.DataFrame(neg_hyps)

In [106]:
len(pos_hyps), len(neg_hyps)

(731, 780)

In [58]:
len(pos_hyps), len(neg_hyps)

(31, 51)

In [39]:
pos_hyps = pd.DataFrame(pos_hyps)
neg_hyps = pd.DataFrame(neg_hyps)

In [40]:
pos_hyps.shape, neg_hyps.shape

((761, 10), (789, 10))

In [112]:
neg_hyps.to_csv('hypothesises/test_neg_hyps.csv', header=True, index=False)

In [28]:
ls hypothesises/

test_neg_hyps.csv  test_pos_hyps.csv


In [31]:
pos_hyps = pd.read_csv('hypothesises/test_pos_hyps.csv')
neg_hyps = pd.read_csv('hypothesises/test_neg_hyps.csv')
pos_hyps.shape, neg_hyps.shape

((761, 10), (789, 10))

In [69]:
# count = 0
# for i, obj in trainX.iterrows():
#     count += 1
#     print(i, type(obj))
#     if count > 5:
#         break

In [293]:
# for i, obj in test_sample.iterrows():
#     for key, val in obj.iteritems():
#         print(key, type(val[0]), type(val[1]))

In [292]:
# for i, pos_obj in pos_hyps.iterrows():
#     for key, val in obj.iteritems():
#         print(key, type(val[0]), type(val[1]))
#     break

In [69]:
some_func = lambda x, y: (min(x[0], y[0]), max(x[1], y[1]))

In [291]:
# pos_hyps.infer_objects().info()

In [109]:
for i, obj in test_sample.iterrows():
    for i, pos_obj in pos_hyps.iterrows():
        repres = utils.similarity(obj, pos_obj)
        break

In [143]:
%%time
t = hypothesises_to_feat_matrix(pos_hyps=pos_hyps, neg_hyps=neg_hyps, trainX=train_pos.sample(1))

CPU times: user 1.67 s, sys: 9.07 ms, total: 1.68 s
Wall time: 1.68 s


In [168]:
pd.DataFrame(np.zeros(shape=(1, 12)), index=[12], columns=range(12))

,0,1,2,3,4,5,6,7,8,9,10,11
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [192]:
pd.DataFrame(np.zeros(shape=(1, 12)))

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [223]:
test_sample.iloc[0].name

113187

In [276]:
def to_binary_repr(num: int, pos_hyps: pd.DataFrame, neg_hyps: pd.DataFrame, indices: list, trainX: pd.DataFrame):
    ind = indices[num]
    obj = trainX.loc[ind]
    pos_features = [f'pos_feat_{feat_num}' for feat_num in pos_hyps.index]
    neg_features = [f'neg_feat_{feat_num}' for feat_num in neg_hyps.index]
    features = pos_features + neg_features
    start_values = np.zeros(shape=(1, len(features)))
#     print(start_values.shape)
#     print(obj.name)
    result = pd.DataFrame(data=start_values,index=[obj.name], columns=features, dtype='int')
    ind = obj.name
    for pi in range(pos_hyps.shape[0]):
        feat_repr = utils.similarity(obj, pos_hyps.iloc[pi])
        is_included = pos_hyps.iloc[pi].equals(feat_repr)
        if is_included:
            result.loc[ind, f'pos_feat_{pi}'] = 1
        else:
            result.loc[ind, f'pos_feat_{pi}'] = 0
        
    for pi in range(neg_hyps.shape[0]):
        feat_repr = utils.similarity(obj, neg_hyps.iloc[pi])
        is_included = neg_hyps.iloc[pi].equals(feat_repr)
        if is_included:
            result.loc[ind, f'neg_feat_{pi}'] = 1
        else:
            result.loc[ind, f'neg_feat_{pi}'] = 0
                          
    return result

def transform_to_feature_matrix(pos_hyps: pd.DataFrame, neg_hyps: pd.DataFrame, trainX: pd.DataFrame, n_jobs: int = 4):
    
    indices = trainX.index
    transform_func = partial(to_binary_repr, pos_hyps=pos_hyps, 
                             neg_hyps=neg_hyps, indices=indices, trainX=trainX)
    
    with ProcessPoolExecutor(max_workers=n_jobs) as executor:
        obj_features = executor.map(transform_func, range(len(indices)))

    features = pd.concat(obj_features)#pd.DataFrame(obj_features)
    
    return features

In [290]:
%%time
gen = transform_to_feature_matrix(pos_hyps=pos_hyps, neg_hyps=neg_hyps, trainX=trainX.sample(512))

CPU times: user 4.07 s, sys: 442 ms, total: 4.51 s
Wall time: 3min 47s


In [285]:
1.18, 1.25, 1.71, 2 range 1 4
3.95(8), 7.35(16), 29(64), 512

(1.18, 1.25, 1.71)

In [248]:
t.sum(axis=1)

28125    3
dtype: int64

In [140]:
def hypothesises_to_feat_matrix(pos_hyps: pd.DataFrame, neg_hyps: pd.DataFrame, trainX: pd.DataFrame):
    pos_features = [f'pos_feat_{feat_num}' for feat_num in pos_hyps.index]
#     neg_features = [f'neg_feat_{feat_num}' for feat_num in neg_hyps[pi]s.index]
    
    result = pd.DataFrame(index=trainX.index, columns=pos_features)# + neg_features)
    
    for i in range(trainX.shape[0]):
        for pi in range(pos_hyps.shape[0]):

            feat_repr = utils.similarity(obj, pos_hyps.iloc[pi])
            is_included = pos_hyps.iloc[pi].equals(feat_repr)
            if is_included:
                result.loc[i, f'pos_feat_{pi}'] = 1
            else:
                result.loc[i, f'pos_feat_{pi}'] = 0
                
        for pi in range(neg_hyps.shape[0]):
            feat_repr = utils.similarity(obj, neg_hyps.iloc[pi])
            is_included = neg_hyps.iloc[pi].equals(feat_repr)
            if is_included:
                result.loc[i, f'neg_feat_{pi}'] = 1
            else:
                result.loc[i, f'neg_feat_{pi}'] = 0
    
    
    return result
    

In [ ]:
###Данный вариант не прокатит, слишком долго
### Нужно придумать другой вариант

In [70]:
# %%time
# result = hypothesises_to_feat_matrix(pos_hyps=pos_hyps, neg_hyps=neg_hyps, trainX=trainX.sample(1000))

In [62]:
# a = pd.Series({'feat1':(1,1), 'feat2':(0, 1.4), 'feat3':(3, 4)})
# b = pd.Series({'feat1':(1,2), 'feat2':(1, 1.4), 'feat3':(7, 7)})
# c = pd.Series({'feat1':(1,1), 'feat2':(0.8, 1.6), 'feat3':(3, 4)})
# d = pd.Series({'feat1':(2,3), 'feat2':(0, 1.4), 'feat3':(4, 6)})


# print(a.equals(b))
# print(a == b)

# test1 = pd.Series({'feat1':(1,3), 'feat2':(1, 3.4), 'feat3':(1, 1.9)})
# test2 = pd.Series({'feat1':(2,2.2), 'feat2':(2, 2.4), 'feat3':(3, 3.9)})
# test3 = pd.Series({'feat1':(3,3), 'feat2':(4, 4.4), 'feat3':(4, 4.9)})

g1 = pd.Series({'feat1':(1,1), 'feat2':(1.5, 1.5)})
g2 = pd.DataFrame({'feat1':(-1,-1), 'feat2':(0, 0)})
g3 = pd.Series({'feat1':(0.5,0.5), 'feat2':(1, 1)})

test1 = pd.Series({'feat1':(0.1, 1), 'feat2':(-0.5, 0)})
test2 = pd.DataFrame({'feat1':(-0.1, 1), 'feat2':(0.5, 0.8)})
tst = pd.Series({'feat1':(-1, 1), 'feat2':(0, 1.5)})

In [287]:
# for i,obj in sample.iterrows():
#     print(similarity(pd.DataFrame(d).T, obj))

In [286]:
# sample = pd.DataFrame([g1, g2, g3, test1, test2])
# sample

In [248]:
g1

feat1        (1, 1)
feat2    (1.5, 1.5)
dtype: object

In [95]:
type(d), type(sample)

(pandas.core.series.Series, pandas.core.frame.DataFrame)

In [186]:
# is_included_in_repr(d, sample)

Тестовая функция провекри np.random.choice

In [40]:
def compute_stat(times, data):
    print(f"times = {times}")
    sample = np.random.RandomState().choice(data, replace=False, size=10)
    print(f"sample = {sample}")
    sample_sum = np.sum(sample)
    print(f"sample sum = {sample_sum}")
    return sample_sum

def run_computing(n_jobs=2):
    data = list(range(100))
    
    compute_func = partial(compute_stat, data=data)
    
    with ProcessPoolExecutor(max_workers=n_jobs) as executor:
        results = executor.map(compute_func, list(range(26)))
    
    return results

In [284]:
# t = run_computing(n_jobs=6)